In [1]:
#Import Libraries:
import os
from time import sleep
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
import requests
import pickle

In [2]:
def get_infos(house_soup):
    location = house_soup.find(class_ = "address").get_text()[2:-2]
    features = house_soup.find_all(class_ = "value")
    house_info = dict()
    house_info['Location'] = location
    for idx in range(len(features)):
        house_info['Feature ' + str(idx + 1)] = features[idx].get_text()[2:-2]
    return house_info

In [3]:
page = 1
house_infos = list()
while True:
    url = requests.get("https://sosanhnha.com/search?iCat=57&iCitId=13326&iDisId=0&iWardId=0&iPrice=0&keyword=&page=" + str(page))
    page_soup = bs(url.text, 'html.parser')
    try:
        page_soup.find("b").get_text() == "Không tìm thấy kết quả nào phù hợp"
        break
    except:
        pass
    houses = page_soup.find_all(class_ = "name", href=True)
    if (houses == []): break
    for house in houses:
        house_page = requests.get("https://sosanhnha.com/" + house['href'])
        house_soup = bs(house_page.text, 'html.parser')
        try: 
            house_infos.append(get_infos(house_soup))
        except: continue
    with open('save_page.txt','w') as file:
        file.write(str(page))
        file.close()
    if (page % 10 == 0):
        with open('../Raw_Data/sosanhnha.pickle','wb') as handle:
            pickle.dump(house_infos, handle, protocol=pickle.HIGHEST_PROTOCOL)
    page+=1
if (house_infos):
    with open('../Raw_Data/sosanhnha.pickle','wb') as handle:
        pickle.dump(house_infos, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [4]:
with open("../Raw_Data/sosanhnha.pickle","rb") as handle:
    dataset = pickle.load(handle)
dataset = pd.DataFrame(data = dataset)
dataset = dataset.drop(columns = ['Feature 3','Feature 4','Feature 5','Feature 6'])
dataset = dataset.rename(columns = {'Feature 1' : 'Price', 'Feature 2' : 'Area'})
dataset

,Location,Price,Area
0,"Phố Tôn Đức Thắng, Phường Hòa Khánh Nam, Liên ...",3 triệu/tháng,55 m²
1,"Đường Núi Thành, Phường Hòa Thuận Đông, Quận H...","2,2 triệu/tháng",20 m²
2,"Kiệt 292 Hải Phòng, Phường Tam Thuận, Quận Tha...",3 triệu/tháng,35 m²
3,"Đường Nguyễn Chánh, Phường Hòa Khánh Bắc, Quận...","1,2 triệu/tháng",20 m²
4,"Đường Tô Hiến Thành, Phường Phước Mỹ, Quận Sơn...","1,1 triệu/tháng",5 m²
...,...,...,...
360,Núi Thành,"4,8 triệu/tháng",30 m²
361,"Núi Thành, Phường Hòa Cường Bắc, Quận Hải Châu...","4,8 triệu/tháng",30 m²
362,"148, Đường Ngũ Hành Sơn, Phường Mỹ An, Quận Ng...","1,7 triệu/tháng",20 m²
363,"Đường Tiên Sơn 6, Phường Hòa Cường Nam, Quận H...",5 triệu/tháng,40 m²


In [ ]:
dataset.to_csv('../Raw_Data/sosanhnha.csv')